In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import IntegerType

In [2]:
# change to be reflective of your environment
data_dir = '/home/cole/Workspace/School/Capstone/data/first_data_set/TestData/'

In [3]:
# initiates a SparkContext which is necessary for accessing data in Spark
sc = SparkContext()
sqlContext = SQLContext(sc)
# change to match your environment
output_dir = data_dir + "/merge_data"

In [4]:
# based around linking to BACKUP_OBJECTS
CSV_Merge_List = {
    'NODES' : 'NODEID',
    'FILESPACES' : 'FSID',
    'AF_BITFILES' : ['OBJID', 'BFID'],
    'SD_RECON_ORDER' : 'OBJID',
    'SS_POOLS' : 'POOLID',
    'SD_CHUNK_LOCATIONS' : 'POOLID',}

In [5]:
merge = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/BACKUP_OBJECTS.csv"])

for key, value in CSV_Merge_List.items():
    temp = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"/" + key + ".csv"])
    if len(value) == 2:
            merge = merge.join(temp, merge[value[0]] == temp[value[1]], how='left')
    else:        
        merge = merge.join(temp, [value], how='left')

In [6]:
df = merge
df = df.filter(df.POOLID. isNotNull())
df = df.withColumn("POOLID", df["POOLID"].cast("int"))
df = df.filter(df.ATTRLENGTH. isNotNull())
df = df.withColumn("ATTRLENGTH", df["ATTRLENGTH"].cast("float"))
df = df.filter(df.BFSIZE. isNotNull())
df = df.withColumn("BFSIZE", df["BFSIZE"].cast("float"))
df = df.filter(df.HDRSIZE. isNotNull())
df = df.withColumn("HDRSIZE", df["HDRSIZE"].cast("float"))
df = df.filter(df.OBJID. isNotNull())
df = df.withColumn("OBJID", df["OBJID"].cast("float"))
df.count()

921346

In [7]:
df.select("OBJID", "ATTRLENGTH", "BFSIZE", "HDRSIZE", "POOLID").write.options(header='true').format('com.databricks.spark.csv').save(data_dir + "/merge_data/4_features")

In [8]:
df.count()

921346

In [9]:
new_df = sqlContext.read.format('com.databricks.spark.csv').option("header", "true").load([data_dir+"//merge_data/4_features/*.csv"])

In [10]:
new_df.count()

921346

In [11]:
sc.stop()